In [4]:
# Simple LLM prompt 

import time 

from vertexai import init 

from vertexai.generative_models import GenerativeModel, Part, GenerationConfig 

st = time.time() 

init(project="cjnyasg-customer360", location="us-central1")  

model = GenerativeModel("gemini-2.0-flash-001") 

prompt = 'Create a list in python format of top 5 AMD ryzen 7 desktop CPU. No explanation is needed.' 

input_parts = [Part.from_text(prompt)]  

generation_config = GenerationConfig(temperature=0, max_output_tokens=8192) 

response = model.generate_content(contents=input_parts, generation_config=generation_config) 

response = response.text 

print(response) 

print(time.time() - st) 

```python
top_5_ryzen_7 = [
    "AMD Ryzen 7 7800X3D",
    "AMD Ryzen 7 5800X3D",
    "AMD Ryzen 7 7700X",
    "AMD Ryzen 7 5700X",
    "AMD Ryzen 7 5700G"
]
```

1.7703123092651367


Activity 1

In [8]:
import time
import fitz  # PyMuPDF
from PIL import Image
import io

from vertexai import init
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

def extract(pdf_path: str, page_number: int = 0):
    """Extract prompt text and the first image from a PDF."""
    doc = fitz.open(pdf_path)
    page = doc[page_number]
    prompt_text = page.get_text().strip()

    images = page.get_images(full=True)
    if not images:
        raise ValueError("No images found on the specified page.")
    
    # Extracts first image from the page
    xref = images[0][0]
    base_image = doc.extract_image(xref)
    image_bytes = base_image["image"]
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    return prompt_text, image

def describe_image(pdf_path: str, page_number: int = 0):
    st = time.time()
    init(project="cjnyasg-customer360", location="us-central1")

    model = GenerativeModel("gemini-2.0-flash-001")

    prompt_text, image = extract(pdf_path, page_number)

    # Convert PIL image to bytes
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_bytes = img_byte_arr.getvalue()

    # Create Gemini-compatible Parts
    image_part = Part.from_data(data=img_bytes, mime_type="image/png")
    prompt_part = Part.from_text(prompt_text)

    input_parts = [prompt_part, image_part]

    generation_config = GenerationConfig(temperature=0.2, max_output_tokens=1024)

    response = model.generate_content(contents=input_parts, generation_config=generation_config)
    description = response.text.strip()

    result = {"Description": description}
    print("Time elapsed:", time.time() - st)
    print(result)
    print(type(result))

    return result

# === Usage ===
if __name__ == "__main__":
    describe_image("C:/Users/richard.c.ferrer/Agents/image_ai.pdf") 


Time elapsed: 4.346190929412842
{'Description': "The image shows a brown dachshund dog standing on its hind legs in front of a black electric piano. The dog is wearing a black collar with silver accents. Its front paws are resting on the piano keys, as if it is playing the instrument. Sheet music is placed on the piano's music stand. The background is blurred, but it appears to be an indoor setting with a plant visible behind the piano. The overall impression is humorous, suggesting the dog is a musician or is pretending to play the piano."}
<class 'dict'>


Activity 2

In [7]:
import time
import csv

from vertexai import init
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

# Start timer
st = time.time()

# Initialize Vertex AI
init(project="cjnyasg-customer360", location="us-central1")

# Initialize model
model = GenerativeModel("gemini-2.0-flash-001")

# Prompt: request CSV format only
prompt = (
    "Generate a CSV with two columns: AMD Ryzen and Intel. "
    "Each column should contain 5 current desktop CPU models. "
    "Output only the CSV content, no explanation."
)

# Send prompt
input_parts = [Part.from_text(prompt)]
generation_config = GenerationConfig(temperature=0, max_output_tokens=8192)
response = model.generate_content(contents=input_parts, generation_config=generation_config)
response_text = response.text.strip().replace("```csv", "").replace("```", "").strip()

# Print response (for debugging or display)
print(response_text)

# Save to CSV file
csv_filename = "C:/Users/richard.c.ferrer/Agents/cpu_models.csv"
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for line in response_text.splitlines():
        row = [cell.strip() for cell in line.split(",")]
        writer.writerow(row)

print(f"\nCSV saved as: {csv_filename}")
print("Time elapsed:", time.time() - st)


AMD Ryzen,Intel
Ryzen 9 7950X3D,Core i9-14900K
Ryzen 7 7700X,Core i7-14700K
Ryzen 5 7600X,Core i5-14600K
Ryzen 5 5600X,Core i5-13400F
Ryzen 3 4100,Core i3-12100F


PermissionError: [Errno 13] Permission denied: 'C:/Users/richard.c.ferrer/Agents/cpu_models.csv'